In [3]:
#Import libraries
import pandas as pd
import psycopg2

In [4]:
#Connect to the database
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [5]:
#Run a test query
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

/tmp/ipython-input-3105154302.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [6]:
df.columns

Index(['dbn', 'school_name', 'borough', 'building_code', 'phone_number',
       'fax_number', 'grade_span_min', 'grade_span_max', 'expgrade_span_min',
       'expgrade_span_max',
       ...
       'number_programs', 'Location 1', 'Community Board', 'Council District',
       'Census Tract', 'Zip Codes', 'Community Districts',
       'Borough Boundaries', 'City Council Districts', 'Police Precincts'],
      dtype='object', length=105)

In [7]:
query = "SELECT * FROM nyc_schools.school_demographics s WHERE s.dbn='27Q260' ;"
df = pd.read_sql(query, conn)
df.columns

/tmp/ipython-input-266311014.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Index(['dbn', 'Name', 'schoolyear', 'fl_percent', 'frl_percent',
       'total_enrollment', 'prek', 'k', 'grade1', 'grade2', 'grade3', 'grade4',
       'grade5', 'grade6', 'grade7', 'grade8', 'grade9', 'grade10', 'grade11',
       'grade12', 'ell_num', 'ell_percent', 'sped_num', 'sped_percent',
       'ctt_num', 'selfcontained_num', 'asian_num', 'asian_per', 'black_num',
       'black_per', 'hispanic_num', 'hispanic_per', 'white_num', 'white_per',
       'male_num', 'male_per', 'female_num', 'female_per'],
      dtype='object')

In [8]:
query = "SELECT s.ell_percent FROM nyc_schools.school_demographics s WHERE s.dbn='27Q260' ;"
df = pd.read_sql(query, conn)
df.head()

/tmp/ipython-input-3198710681.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ell_percent


In [9]:
df.columns

Index(['ell_percent'], dtype='object')

In [10]:
#Question 1. School Distribution: How many schools are there in each borough?
query = """
SELECT
	hsd.borough,
	COUNT (distinct hsd.dbn) as number_of_schools
FROM nyc_schools.high_school_directory hsd
group by hsd.borough
order by number_of_schools desc

"""

df = pd.read_sql(query, conn)
df

/tmp/ipython-input-1206658039.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,number_of_schools
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


In [11]:
#Question 2. Language Learners: What is the average % of English Language Learners (ELL) per borough?
query = """
select
	AVG(sd.ell_percent ) as average_percent_ELL,
	hsd.borough
from nyc_schools.school_demographics sd
right join nyc_schools.high_school_directory hsd
join nyc_schools.school_safety_report ssr on ssr.dbn = hsd.dbn
on sd.dbn  = hsd.dbn
group by hsd.borough
order by average_percent_ELL desc

"""

df = pd.read_sql(query, conn)
df

/tmp/ipython-input-4266467985.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,average_percent_ell,borough
0,NaN,Brooklyn
1,NaN,Queens
2,NaN,Staten Island
3,NaN,Bronx
4,7.5725,Manhattan


In [12]:
#Question 3. School supporting special needs: Using the data from the school demographics and high school directory, write a query to find the top 3 schools in each borough with the highest percentage of special education students (sped_percent)
query = """
WITH Ranked_Schools AS (
  SELECT
    h.borough,
    h.school_name,
    s.sped_percent,
    ROW_NUMBER() OVER(PARTITION BY h.borough ORDER BY s.sped_percent DESC) as rank_num
  FROM
    nyc_schools.school_demographics s
    RIGHT JOIN nyc_schools.high_school_directory h ON s.dbn = h.dbn
  WHERE s.sped_percent IS NOT NULL
)
SELECT
  borough,
  school_name,
  sped_percent
FROM
  Ranked_Schools
WHERE
  rank_num <= 3
ORDER BY
  borough,
  sped_percent DESC;

"""

df = pd.read_sql(query, conn)
df

/tmp/ipython-input-2606997740.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7


#Insights

In [ ]:
Q1_School Distribution: BBrooklyn has the highest number of high schools (121), followed closely by the Bronx (118).

Q2_Language Learners: Only Manhattan returned ELL data (7.57% average) and the rest of the boroughs did not have any ELL data.

Q3_School supporting special needs: Only top 3 Manhattan schools appeared in special education analysis. Among them, East Side Community School shows highest special education percentages.